Install Dependencies
=============
pip install pandas matplotlib numpy jupter openpyxl nltk gensim pyLDAvis
----------

In [3]:
# Importing require library
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import itertools
import datetime
import time
import datetime
import nltk
from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer
import string
import gensim
from gensim import corpora
import pyLDAvis.gensim
# downloading component of nltk 
nltk.download('stopwords')
nltk.download('wordnet')

# Loading csv data file
df = pd.read_csv('data.csv')
df.head()

C:\Anaconda3\lib\site-packages\sklearn\decomposition\nmf.py:972: DeprecationWarning: invalid escape sequence \s
  """
C:\Anaconda3\lib\site-packages\sklearn\decomposition\nmf.py:1186: DeprecationWarning: invalid escape sequence \s
  """
C:\Anaconda3\lib\site-packages\sklearn\manifold\t_sne.py:420: DeprecationWarning: invalid escape sequence \s
  """


[nltk_data] Downloading package stopwords to C:\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package wordnet to C:\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,Unnamed: 0,author,publicationTime,bodyText
0,0,2.209799e+09,10/02/2016 10:30,RT @350Australia: Adani Group's Aust #coal min...
1,1,2.209799e+09,10/02/2016 10:36,RT @avivaimhof: Poor old #coal. Now even #Viet...
2,2,4.306011e+07,10/02/2016 10:37,RT @market_forces: Funds have burned billions ...
3,3,2.209799e+09,10/02/2016 10:37,RT @avivaimhof: #Vietnam PM Announces Retreat ...
4,4,5.855606e+08,10/02/2016 10:41,RT @350Australia: Adani Group's Aust #coal min...


In [4]:
# Convert string datetime into python datetime object
df["publicationTime"] = pd.to_datetime(df["publicationTime"],  format='%d/%m/%Y %H:%M')
df['bodyText'].fillna('not found',inplace=True)

In [5]:
# Enter date and should enter format DD-MM-YYYY and when you enter using single quote both side on date
# Example: '10-02-2016' 
date_entry = input("Enter a date in DD-MM-YYYY format and user single quote like: '10-02-2016'")
day, month, year = map(int, date_entry.split('-'))
date = datetime.date(year, month, day)
if not (df.iloc[0]['publicationTime'].date() <= date and df.iloc[-1]['publicationTime'].date() >= date):
    raise Exception("Error: Date should be between tweets data")

Enter a date in DD-MM-YYYY format and user single quote like: '10-02-2016'11-02-2016


In [6]:
# Enter Interval in seconds that will show each interval related data
interval = input('Enter Interval in seconds : ')
dateslot_df = df.loc[df['publicationTime'].apply(lambda x: x.date()) == date]
dateslot_df.head()

Enter Interval in seconds : 1500


,Unnamed: 0,author,publicationTime,bodyText
263,263,2.698154e+08,2016-02-11 00:01:00,Best Minister in the World is proud part of #T...
264,264,2.386230e+09,2016-02-11 00:08:00,RT @market_forces: Funds have burned billions ...
265,265,2.698154e+08,2016-02-11 00:14:00,THE WORLD'S BEST MINISTER has proven track rec...
266,266,8.990323e+07,2016-02-11 00:23:00,Select Committee on Unconventional Gas Mining ...
267,267,6.112781e+08,2016-02-11 00:27:00,Indian media reporting massive new (unbankable...


In [15]:
# creating new DataFrame of custom data data store count of tweet, users, and number of tweets
users = []
combine_data = []
counter  = 0
tweet_time = 0
old_timestamp = 0
row_count = 0
df_tweet = pd.DataFrame()
tweet_list = []
index_counter = 0
def filter_data(text, combine_data):
    for t in text.split(' '):
        if t.startswith('@'):
            combine_data.append(t)
    return combine_data

def filter_retweet(text, count):
    for index, t in enumerate(text.split(' ')):
        if index == 0 and t == 'RT':
            count += 1
    return count  
for i, row in dateslot_df.iterrows():
    is_exist = pd.isnull(row['publicationTime'])
    if not is_exist:
        timestamp = time.mktime(row['publicationTime'].timetuple())
        if i == 0:
            old_timestamp = timestamp
        if tweet_time > old_timestamp + float(interval):
            diff = (tweet_time - old_timestamp + float(interval))/float(interval)
            for d in range(1,int(diff)):
                index_counter += 1
                old_timestamp = old_timestamp+(float(interval*1)) 
                pub = datetime.datetime.fromtimestamp(old_timestamp).strftime('%Y-%m-%d %H:%M:%S')
                temp = pd.DataFrame({'publicationTime': pub, 'users': len(set(combine_data)), 'retweet': counter,'count': row_count, 'tweets': str(tweet_list)},index=[index_counter])
                df_tweet = pd.concat([df_tweet, temp])
            combine_data = []
            tweet_list = []
            counter = 0
            row_count = 0
        else:
            combine_data = filter_data(row['bodyText'], combine_data)
            counter = filter_retweet(row['bodyText'], counter)
            tweet_time = timestamp
            tweet_list.append(row['bodyText'])
            row_count += 1
df_tweet.head()

OSError: [Errno 22] Invalid argument

In [8]:
# getting value of action that will help to show graph of that data
column_dict = {1:'users', 2:'count',3:'retweet'}
action = int(input('1 for users, 2 for tweets count, 3 for retweets: '))
plt.bar(df_tweet['publicationTime'], df_tweet[column_dict[action]], align='center')
plt.title('Interval')
plt.show()

1 for users, 2 for tweets count, 3 for retweets: 1


KeyError: 'publicationTime'

In [9]:
# Calculating Velocity
velocity = df_tweet['count'].sum() / float(df_tweet.shape[0]*interval) 
print "Velocity: {} tweets/second".format(velocity)

SyntaxError: invalid syntax (<ipython-input-9-ebb81922a028>, line 3)

In [10]:
def gen_window(df):
    window = []
    count = 1
    for i, row in df.iterrows():
        window.append("Window {}".format(count))
        count += 1
    return window


In [ ]:
# Saving specific interval tweets in csv file
df_tweet['window'] = gen_window(df_tweet)
df_tweet.to_csv('output.csv')

In [ ]:
# Enter Windows number that should be integer and should be present in 
window = input('Enter Windows number : ')
row = df_tweet.loc[df_tweet['window'] == 'Window {0}'.format(window)]


In [ ]:
# Saving all data into xlsx 
data = pd.DataFrame()
tweet_list = []
for index, r in enumerate(eval(row.iloc[0]['tweets'])):
    temp = pd.DataFrame({'tweet':str(r)},index=[index])
    tweet_list.append(r)
    data = pd.concat([data, temp])
writer = pd.ExcelWriter('output.xlsx')
data.to_excel(writer,'Sheet1')
writer.save()

In [ ]:
# performing task of word analysis
stop = set(stopwords.words('english'))
exclude = set(string.punctuation) 
lemma = WordNetLemmatizer()
def clean(doc):
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    return normalized
doc_clean = [clean(doc).split() for doc in tweet_list]  

In [ ]:
# Creating the object for LDA model using gensim library
dictionary = corpora.Dictionary(doc_clean)
doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]
Lda = gensim.models.ldamodel.LdaModel
ldamodel = Lda(doc_term_matrix, num_topics=row.iloc[0]['count'], id2word = dictionary, passes=50)

In [ ]:
# printing all data of LDA model
ldamodel.print_topics(num_topics=row.iloc[0]['count'], num_words=5)

In [ ]:
# Showing visualization of LDA model
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(ldamodel, doc_term_matrix, dictionary)